In [ ]:
# Homework 4
# Center of Mass Position and Velocity
# Write your name here

### Keep in mind this is just a template; you don't need to follow every step and feel free to change anything.
### We also strongly encourage you to try to develop your own method to solve the homework.

In [2]:
# Add path to other HW folders
# Incites modular design / No need to copy ReadFile into Homework3 folder
import sys
sys.path.append("../")

# Import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl
from Homework2.ReadFile import Read

In [4]:
filename = '../../SampleData/MW_000.txt'
ptype = 1
time, total, data = Read(filename)

index = np.where(data['type'] == ptype)

array([(-2.51725e+00,  19.1588 ,  5.28528e+01),
       (-2.86601e+02, 298.455  ,  3.91692e+02),
       (-5.05945e-01, -28.6337 , -8.39565e+01), ...,
       (-3.29432e+00,   3.36725,  1.09023e-01),
       ( 2.57806e-01,   5.31409, -6.62670e-01),
       (-6.57662e-01,   3.32552, -2.51660e+00)],
      dtype={'names':['x','y','z'], 'formats':['<f8','<f8','<f8'], 'offsets':[16,24,32], 'itemsize':64})

In [191]:
class CenterOfMass:
# Class to define COM position and velocity properties of a given galaxy 
# and simulation snapshot
    
    
    def __init__(self, filename, ptype):
        # Initialize the instance of this Class with the following properties:
    
        # Read data in the given file using Read
        self.time, self.total, self.data = Read(filename)                                                                                            

        # Create an array to store indexes of particles of desired Ptype
        self.index = np.where(self.data['type'] == ptype)

        # store the mass, positions, velocities of only the particles of the given type
        # the following only gives the example of storing the mass
        self.m = self.data['m'][self.index] 
        self.x = self.data['x'][self.index]
        self.y = self.data['y'][self.index]
        self.z = self.data['z'][self.index]
        self.r = np.sqrt(self.x**2 + self.y**2 + self.z**2)
        self.vx = self.data['vx'][self.index]
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index]


    def COMdefine(self,x,y,z,m):
    # Function to compute the center of mass position or velocity generically
    # input: array (a,b,c) of positions or velocities and the mass
    # returns: 3 floats  (the center of mass coordinates)

        # write your own code to compute the generic COM using Eq. 1 in the homework instructions
        # xcomponent Center of mass
        Xcom = ( x * m ).sum()/m.sum()
        # ycomponent Center of mass
        Ycom = ( y * m ).sum()/m.sum()
        # zcomponent Center of mass
        Zcom = ( z * m ).sum()/m.sum()
        return Xcom, Ycom, Zcom
    
    
    def COM_P(self, delta):
        # Function to specifically return the center of mass position and velocity                                         
        # input:                                                                                                           
        #        particle type (1,2,3)                                                                                     
        #        delta (tolerance)                                                                                         
        # returns: One vector, with rows indicating:                                                                                                                                                                            
        #       3D coordinates of the center of mass position (kpc)                                                             

        # Center of Mass Position                                                                                      
        ###########################                                                                                    

        # Try a first guess at the COM position by calling COMdefine                                                   
        XCOM, YCOM, ZCOM = self.COMdefine(self.x, self.y, self.z, self.m)
        # compute the magnitude of the COM position vector.
        # write your own code below
        RCOM = np.sqrt( XCOM**2 + YCOM**2 + ZCOM**2 )
        print(f'First RCOM guess\t =', RCOM)

        # iterative process to determine the center of mass                                                            

        # change reference frame to COM frame                                                                          
        # compute the difference between particle coordinates                                                          
        # and the first guess at COM position
        # write your own code below
        xNew = self.x - XCOM
        yNew = self.y - YCOM
        zNew = self.z - ZCOM
        RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)
        
        # find the max 3D distance of all particles from the guessed COM                                               
        # will re-start at half that radius (reduced radius)                                                           
        RMAX = max(RNEW)/2.0

        # pick an initial value for the change in COM position                                                      
        # between the first guess above and the new one computed from half that volume
        # it should be larger than the input tolerance (delta) initially
        CHANGE = 1000.0

        # start iterative process to determine center of mass position                                                 
        # delta is the tolerance for the difference in the old COM and the new one.    
        
        while (CHANGE > delta):
            # select all particles within the reduced radius (starting from original x,y,z, m)
            # write your own code below (hints, use np.where)
            print(f'RMAX\t= {RMAX}')
            
            if not np.any(RNEW < RMAX):
                print(f'\nThere are no particles within RMAX {RMAX:.3f}')
                print(f'RNEW: Min={RNEW.min():.2f}\tMax={RNEW.max():.2f}\tRMEAN={RNEW.mean():.2f}')
                raise ValueError()
            
            index2 = np.where( RNEW < RMAX)
            x2 = xNew[index2]
            y2 = yNew[index2]
            z2 = zNew[index2]
            m2 = self.m[index2]

            # Refined COM position:                                                                                    
            # compute the center of mass position using                                                                
            # the particles in the reduced radius
            # write your own code below
            XCOM2, YCOM2, ZCOM2 = self.COMdefine(x2, y2, z2, m2)
            # compute the new 3D COM position
            # write your own code below
            RCOM2 = np.sqrt( XCOM2**2 + YCOM2**2 + ZCOM2**2 )
            print('iterative RCOM\t =', RCOM2)
            # determine the difference between the previous center of mass position                                    
            # and the new one.
            CHANGE = np.abs(RCOM - RCOM2)
            # print(XCOM2, YCOM2, ZCOM2, RCOM2)
            # uncomment the following line if you wnat to check this                                                                                               
            print ("CHANGE = ", CHANGE)                                                                                     

            # Before loop continues, reset : RMAX, particle separations and COM                                        

            # reduce the volume by a factor of 2 again                                                                 
            RMAX = RMAX/2.0
            # check this.                                                                                              
            print ("maxR", RMAX)                                                                                      

            # Change the frame of reference to the newly computed COM.                                                 
            # subtract the new COM
            # write your own code below
            xNew = self.x - XCOM2
            yNew = self.y - YCOM2
            zNew = self.z - ZCOM2
            
            # xNew = xNew - XCOM2
            # yNew = yNew - YCOM2
            # zNew = zNew - ZCOM2
            RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)

            # set the center of mass positions to the refined values                                                   
            XCOM = XCOM2
            YCOM = YCOM2
            ZCOM = ZCOM2
            # RCOM = np.sqrt( XCOM**2 + YCOM**2 + ZCOM**2 )
            RCOM = RCOM2

        # create a vector to store the COM position                                                                                                                                                       
        COMP = XCOM, YCOM, ZCOM

        # set the correct units usint astropy and round all values
        # and then return the COM positon vector
        # write your own code below
        # return [XCOM, YCOM, ZCOM]
        return np.round(COMP, 2) * u.kpc
        

    def COM_V(self, COMX,COMY,COMZ):
        # Center of Mass velocity
        # input: X, Y, Z positions of the COM
        # returns 3D Vector of COM Velocities
        
        # the max distance from the center that we will use to determine the center of mass velocity                   
        RVMAX = 15.0 * u.kpc

        # determine the position of all particles relative to the center of mass position
        # write your own code below
        xV = self.x * u.kpc - COMX
        yV = self.y * u.kpc - COMY
        zV = self.z * u.kpc - COMZ
        RV = np.sqrt(xV**2 + yV**2 + zV**2)
        
        # determine the index for those particles within the max radius
        # write your own code below
        indexV = RV < RVMAX
        
        if not np.any(indexV):
            print(f'\nThere are no particles within RVMAX {RVMAX:.3f}')
            print(f'RV: Min={RV.min():.2f}\tMax={RV.max():.2f}\tRMEAN={RV.mean():.2f}')
            raise ValueError()

        
        # determine the velocity and mass of those particles within the mas radius
        # write your own code below
        vxnew = self.vx[indexV]
        vynew = self.vy[indexV]
        vznew = self.vz[indexV]
        mnew  = self.m[indexV]
        
        # compute the center of mass velocity using those particles
        # write your own code below
        VXCOM, VYCOM, VZCOM = self.COMdefine(vxnew, vynew, vznew, mnew)

        # create a vector to store the COM velocity
        # set the correct units usint astropy
        # round all values
        # write your own code below
        COMV = np.round([VXCOM, VYCOM, VZCOM], 2) * u.km/u.s

        # return the COM vector                                                                                        
        return COMV

In [192]:
# These are the 'testing' locations of the galaxies data
mw_path = '../../SampleData/MW_000.txt'
m31_path = '../../SampleData/M31_000.txt'
m33_path = '../../SampleData/M33_000.txt'

In [193]:
# Create a Center of mass object for the MW, M31 and M33
# below is an example of using the class for MW
MWCOM  = CenterOfMass(mw_path,  2)
M31COM = CenterOfMass(m31_path, 2)
M33COM = CenterOfMass(m33_path, 2)

In [194]:
# below gives you an example of calling the class's functions
# MW:   store the position and velocity COM
MW_COMP = MWCOM.COM_P(0.1)
MW_COMV = MWCOM.COM_V(MW_COMP[0], MW_COMP[1], MW_COMP[2])
display(MW_COMP, MW_COMV)

First RCOM guess	 = 2.9501930277320705
RMAX	= 22.040962666796222
iterative RCOM	 = 0.3942784931382934
CHANGE =  2.5559145345937773
maxR 11.020481333398111
RMAX	= 11.020481333398111
iterative RCOM	 = 2.731641756135893
CHANGE =  2.3373632629976
maxR 5.5102406666990555
RMAX	= 5.5102406666990555
iterative RCOM	 = 1.0019509966929854
CHANGE =  1.7296907594429078
maxR 2.7551203333495278
RMAX	= 2.7551203333495278
iterative RCOM	 = 1.6770718289446975
CHANGE =  0.6751208322517122
maxR 1.3775601666747639
RMAX	= 1.3775601666747639
iterative RCOM	 = 0.288197662853163
CHANGE =  1.3888741660915345
maxR 0.6887800833373819
RMAX	= 0.6887800833373819
iterative RCOM	 = 0.18107233022934432
CHANGE =  0.10712533262381868
maxR 0.34439004166869097
RMAX	= 0.34439004166869097
iterative RCOM	 = 0.17364990669127478
CHANGE =  0.007422423538069545
maxR 0.17219502083434549


<Quantity [-0.11,  0.13, -0.02] kpc>

<Quantity [-6.54,  1.57, -1.25] km / s>

In [195]:
M31_COMP = M31COM.COM_P(0.1)
M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])
display(M31_COMP, M31_COMV)

First RCOM guess	 = 773.0189990729881
RMAX	= 33.36459887755716
iterative RCOM	 = 0.07153437957891283
CHANGE =  772.9474646934092
maxR 16.68229943877858
RMAX	= 16.68229943877858

There are no particles within RMAX 16.682
RNEW: Min=719.35	Max=830.42	RMEAN=773.13


ValueError: 

In [ ]:
M33_COMP = M33COM.COM_P(0.1)
M33_COMV = M33COM.COM_V(M33_COMP[0], M33_COMP[1], M33_COMP[2])
display(M33_COMP, M33_COMV)